# An Analysis of Wine Production, Consumption, Ratings and Price

### Part I - Wrangling for Wine History Section (MS-Excel Giant File): Cleaning and Transforming Data 
-Database source: Anderson, K. and V. Pinilla (with the assistance of A.J. Holmes), Annual Database of Global Wine Markets, 1835 to 2016, freely available in Excel at the University of Adelaide’s Wine Economics Research Centre, November 2017 www.adelaide.edu.au/wine-econ/databases/global-wine-history
#### The source file has over 100 sheets. For this study we used:
* T6 Volume of wine production
* T8 Volume of wine production per capita
* T10 Volume of wine exports
* T12 Volume of wine exports per capita
* T15 Volume of wine imports
* T17 Volume of wine imports per capita
* T34 Volume of wine consumption
* T38 Volume of wine consumption per capita
* T58 Population

#### The follwoing function works as follows:
* Reads the name of worksheet within the giant Excel Worksheet (string) (required)
* Reads the number of rows to be removed from bottom (brr) and top (thr) (two integers) (optional)
* Removes unnecessary columns (e.g. Other column, Other Asia Pacific, Coeff. of variation)
* Relocate/Rename columns (i.e., Norway and Bel-Lux)
* Takes the year column as index
* Returns the cleaned dataframe with the year as index

In [205]:
#%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmaps
import requests
import json
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot
from statistics import mean
from scipy.stats import sem
from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib as mpl

from api_keys import (owmkey,gkey)
reportimagepath='../reports/images/'

In [4]:
# removing the unnecessary row from top or bottom
# relocating the columns and removing unnecessary ones
# reset indexing
def giant_worksheet(sheet_giant,brr=0, thr=0):
    
    df = pd.read_excel('../data/rawdata/Megafile_of_global_wine_data_1835_to_2016_1217.xlsx', 
                         sheet_name=sheet_giant, header=1, index_col=None)
    df.index.rename('year',inplace=True)
    df=df.reset_index()
    #df.fillna(-1,inplace=True) #fills empty cells with -1
    
    #relocating the Norway column if exist
    try:
        nor = df['Norway']
        df.drop(labels=['Norway'], axis=1,inplace = True)
        df.insert(6, 'Norway', nor)
    except:
        print('NO Norway column found in worksheet:', sheet_giant)

    #dfmf.drop(labels=['nan'], axis=1,inplace = True)
    df=df.rename(columns={'Bel-Lux':'Belgium'}) #chnage the name of Be-Lux to Belgium
    df.drop(labels=['Coeff. of variation'], axis=1,inplace = True) #remove the coeff of var column.
    df.drop(labels=['Other'], axis=1,inplace = True) #remove the Other column
    df.drop(labels=['Other Asia Pacific'], axis=1,inplace = True) #remove the Other column
    #df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

    row_to_drop=list(range(thr))
    df=df.drop(df.index[row_to_drop])
    if brr !=0:
        df=df[:-brr]
    df.set_index('year',inplace=True)
    print('Worksheet: {} wrangling completed!'.format(sheet_giant))
    return df

Pass each worksheet to the giant_worksheet function and save the dataframe to a worksheet of a MS-Excel Workbook called Wine_History.xlsx

In [5]:
outputpath="..\data\cleandata\Wine_History.xlsx"
writer = pd.ExcelWriter(outputpath, engine='xlsxwriter')


#Wine production volume(dfwpv) (KL) (ws #6)
sheetname='T6 Wine production'
dfwpv=giant_worksheet(sheet_giant=sheetname,thr=30)
dfwpv.to_excel(writer, sheet_name=sheetname)

#wine production per capita (dfwppc) (litres) (ws #8)
sheetname='T8 Wine prodn per capita'
dfwppc=giant_worksheet(sheet_giant=sheetname,thr=30,brr=3)
dfwppc.to_excel(writer, sheet_name=sheetname)

#wine production per capita (dfwev) (KL) (ws #10)
sheetname='T10 Wine export vol'
dfwev=giant_worksheet(sheet_giant=sheetname,thr=30,brr=3)
dfwev.to_excel(writer, sheet_name=sheetname)

#Wine import volume(dfwiv) (KL) (ws #15)
sheetname='T15 Wine import vol'
dfwiv=giant_worksheet(sheet_giant=sheetname,thr=30)
dfwiv.to_excel(writer, sheet_name=sheetname)

#Wine consumption volume(dfwcv) (KL) (ws #34)
sheetname='T34 Wine consumption vol'
dfwcv=giant_worksheet(sheet_giant=sheetname,thr=30,brr=2)
dfwcv.to_excel(writer, sheet_name=sheetname)

#Wine consumption per capita (dfwcpc) (litres) (ws #38)
sheetname='T38 Wine consumption per capita'
dfwcpc=giant_worksheet(sheet_giant=sheetname,thr=30,brr=2)
dfwcpc.to_excel(writer, sheet_name=sheetname)

#Global population (dfgp) (*1000) (ws #58)
sheetname='T58 Population'
dfgp=giant_worksheet(sheet_giant=sheetname,thr=30)
dfgp.to_excel(writer, sheet_name=sheetname)


#Wine consumption intensity index (dfwcii) (ws #96)
sheetname='T96 Wine cons intensity index'
dfwcii=giant_worksheet(sheet_giant=sheetname, brr=15,thr=126)
dfwcii.to_excel(writer, sheet_name=sheetname)

writer.save()

NO Norway column found in worksheet: T6 Wine production
Worksheet: T6 Wine production wrangling completed!
NO Norway column found in worksheet: T8 Wine prodn per capita
Worksheet: T8 Wine prodn per capita wrangling completed!
NO Norway column found in worksheet: T10 Wine export vol
Worksheet: T10 Wine export vol wrangling completed!
Worksheet: T15 Wine import vol wrangling completed!
Worksheet: T34 Wine consumption vol wrangling completed!
Worksheet: T38 Wine consumption per capita wrangling completed!
Worksheet: T58 Population wrangling completed!
Worksheet: T96 Wine cons intensity index wrangling completed!



### Part II - Wrangling for Analyses of Ratings, Varieties, Origin and Pricen (CSV File, 130K): Cleaning and Transforming Data 

# Part II - Analyzing Wine Price/Rating/Variety (Working on 130K CSV File)

### Data Wrangling- Part II
#### 1) Import data from 130k Excel workbook
#### 2) Cleaning up the data (e.g. remow null values, tester twitter column),  and 
#### 3) Export the data back to another csv file
#### Note: It reduced the number of studied wines from 129970 to 120916

In [25]:
file_one = "../data/rawdata/winemag-data-130k-v2.csv"
df = pd.read_csv(file_one,index_col=0)
#cleaning data: delete the taster_tweeter address column
df.drop(columns=['taster_twitter_handle'])
#drop any null values
df = df.dropna(subset=['country','points','price'])
df.sort_values(by=['price'],ascending=False)
df.to_csv(path_or_buf="..\data\cleandata\Wine_price_rating_variety.csv")

## Categorize more than 700 wine types to 9 majors and 77 sub-categories:
1) Bold Red: Malbec, Syrah, Shiraz, Mourvedre, Pinotage, Petite Sirah, Touriga Nacional, Cabernet Sauvignon, Bordeaux Blend, Meritage

2) Medium Red: Meriot, Sangiovese, Zinfandel, Cabernet Franc, Tempranillo, Nebbiolo, Barbera, Cotes du Rhone Blend

3) Light Red: Pinot Noir, Grenache, Gamay, St. Laurent, Carignan, Counoise

4) Rich White: Chardonnay, Semillon, Viognier, Marsanne, Roussanne

5) Light White: Sauvignon Blanc, Albarino, Pitot Blanc, Vermentino, Melon de Bourgogne, Gargenega, Trebbiano, Pinot Gris, Pinot Grigio, Veltliner

6) Rose: Provencal Rose, White Zinfandel, Loire Valley Rose, Pinot Noir Rose, Syrah Rose, Garnache Rosado, Bandol Rose, Tempranilio Rose, Saignee Method Rose

7) Sweet White: Moscato, Riesling, Chenin Blanc, Gewurztraminer, Late Harvest Whites, Alascian Pinot Gris

8) Sparkling: Champagne, Prosecco, Cremant, Cava, Metodo Classico, Sparkling Wine, Sparkling Rose

9) Dessert: Port, Sherry, Maderia, Vin Santo, Muscat, PX, Pedro Ximenez

In [125]:
Wine_types={'Bold Red': ['Malbec', 'Syrah','Red Blend','Shiraz', 'Mourvedre', 'Merlot','Bordeaux-style Red Blend', 'Pinotage', 'Petite Sirah', 'Touriga Nacional', 'Cabernet Sauvignon', 'Portuguese Red', 'Meritage'],
            'Medium Red': ['Meriot', 'Sangiovese', 'Rhône-style Red Blend','Zinfandel','Cabernet Franc', 'Tempranillo', 'Nebbiolo', 'Barbera', 'Cotes du Rhone Blend'],
            'Light Red':['Pinot Noir', 'Grenache', 'Gamay', 'St. Laurent', 'Carignan', 'Counoise'],
            'Rich White': ['Chardonnay', 'Semillon','Viognier', 'Marsanne', 'Roussanne'],
            'Light White': ['Bordeaux-style White Blend','Sauvignon Blanc', 'White Blend' , 'Albarino', 'Pitot Blanc', 'Vermentino', 'Melon de Bourgogne', 'Gargenega', 'Trebbiano', 'Pinot Gris', 'Pinot Grigio', 'Veltliner'],
            'Sweet White': ['Moscato', 'Riesling', 'Chenin Blanc', 'Gewurztraminer', 'Late Harvest Whites', 'Alascian Pinot Gris'],
            'Rosé': ['Rosé','Provencal Rose', 'White Zinfandel', 'Loire Valley Rose', 'Pinot Noir Rose', 'Syrah Rose', 'Garnache Rosado', 'Bandol Rose', 'Tempranilio Rose', 'Saignee Method Rose'],
            'Sparkling': ['Champagne', 'Prosecco', 'Cremant', 'Cava', 'Metodo Classico', 'Sparkling Wine', 'Sparkling Rose', 'Sparkling Blend', 'Champagne Blend'],
            'Dessert': ['Port', 'Sherry', 'Maderia', 'Vin Santo', 'Muscat', 'PX', 'Pedro Ximenez']
           }
Wine_types.keys()
for x in Wine_types.keys():
    for y in range(len(Wine_types[x])):
        print('type=',x,'subtype=',Wine_types[x][y])
        A=df[df.variety.str.contains(Wine_types[x][y],na=False)].index.tolist()
        df.loc[A,'vt']=x
idx = df['vt'].isnull( )
df.loc[idx,'vt']='Others'
df.to_csv(path_or_buf="..\data\cleandata\Wine_price_rating_variety.csv")
df.head()

type= Bold Red subtype= Malbec
type= Bold Red subtype= Syrah
type= Bold Red subtype= Red Blend
type= Bold Red subtype= Shiraz
type= Bold Red subtype= Mourvedre
type= Bold Red subtype= Merlot
type= Bold Red subtype= Bordeaux-style Red Blend
type= Bold Red subtype= Pinotage
type= Bold Red subtype= Petite Sirah
type= Bold Red subtype= Touriga Nacional
type= Bold Red subtype= Cabernet Sauvignon
type= Bold Red subtype= Portuguese Red
type= Bold Red subtype= Meritage
type= Medium Red subtype= Meriot
type= Medium Red subtype= Sangiovese
type= Medium Red subtype= Rhône-style Red Blend
type= Medium Red subtype= Zinfandel
type= Medium Red subtype= Cabernet Franc
type= Medium Red subtype= Tempranillo
type= Medium Red subtype= Nebbiolo
type= Medium Red subtype= Barbera
type= Medium Red subtype= Cotes du Rhone Blend
type= Light Red subtype= Pinot Noir
type= Light Red subtype= Grenache
type= Light Red subtype= Gamay
type= Light Red subtype= St. Laurent
type= Light Red subtype= Carignan
type= Light R

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,vt
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,$15.00,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Dessert
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,$14.00,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Light White
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,$13.00,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Sweet White
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,$65.00,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Light Red
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,$15.00,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,Medium Red
